In [2]:
from __future__ import print_function

import argparse
import os 
import random
import torch
import torch.nn as nn
import torch.nn.parallel 
import torch.backends.cudnn as cudnn
import torch.optim as optim 
import torch.utils.data 
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils 

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation 
from IPython.display import HTML

manualSeed = 999
random.seed(manualSeed)
torch.manual_seed(manualSeed)

#### Set base parameters

In [20]:
dataroot = '../../../Downloads/img_align_celeba/'
workers = 2 # Number of workers for dataloader
batch_size = 128 
image_size = 64
nc = 3 # Number of channels in the training images
nz = 100 # Size of z latent vector
ngf = 64 # Size of feature maps in generator
ndf = 64 # Size of feature maps in discriminator
num_epochs = 5 
beta1 = 0.5
ngpu = 1

This is an importnat step because we will be using the ImageFolder dataset class, <br>
which requires there to be subdirectories in the dataset's root folder. <br>
Now, we can create the dataset, create the dataloader, set the device to run on, 
and finally visualize some of the training data.<br>

In [19]:
# Create the dataset
dataset = dset.ImageFolder(root = dataroot,
                           transform = transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                           ]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Device which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu==0) else 'cpu')



RuntimeError: Found 0 files in subfolders of: ../../../Downloads/img_align_celeba
Supported extensions are: .jpg,.jpeg,.png,.ppm,.bmp,.pgm,.tif,.tiff,.webp

In [8]:
# Weight Initialize function

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
# Generator Code 

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf*8, 4, 1, 0, bias=False)
        )
        
    def forward(self, input):
        